This notebook is to test the functionalities without having to run the whole script repeatedly.

It automatically extracts the contents of a Word document, according to the given format, saves the extracted information into dictionary variables, which are pickled and stored locally.

In [2]:
from docx.text.paragraph import Paragraph
from docx.document import Document
from docx.table import _Cell, Table
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl
import docx
from openai import OpenAI
import pickle

iter_bock_items() is the function used to iterate over paragraphs and tables in a docx document

In [3]:
def iter_block_items(parent):
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        raise ValueError("something's not right")

    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            #yield Paragraph(child, parent)
            current_p=Paragraph(child,parent)
            yield current_p
        elif isinstance(child, CT_Tbl):
            table = Table(child, parent)
            for row in table.rows:
                for cell in row.cells:
                    yield from iter_block_items(cell)

Read the desired Word document
(Can't run if the document is open)

In [5]:
doc = docx.Document('Projectreports/23-1111_JT Projektreport.docx')

Initialise Dictionaries that will store the information from the Word document (RN: untill chapter 5: Macht und Interessen)

Infos that do not belong to any Chapter:

In [6]:
#Erste Tabelle Titelseite
D_Titelseite={'Projekt':'',
              'Teilnehmer':'',
              'Firma':'',
              'Adresse':'',
              'Telefon':'',
              'E-Mail':'',
              'Kurs':''}
#D_Projektname={'Projektname':''}

#Änderungshistorie Tabelle
D_Änderungshistorie={'1-Version':'',
                     '1-Datum':'',
                     '1-Ersteller':'',
                     '1-Grund':''} #Further entries are added later on 

#I name this entries chapter 0
D_Chapt_0={'Angaben zum Projekt':'',
           'Eigene Position':''}




Infos from chapt 1. Strategie 4.5.1.

In [7]:
D_Strategie={'Beschreibung des Business Case zum Projekt':'',
                 'Nennung der kritischen Erfolgsfaktoren des Projekts':''}

Infos from chapt 2. Governance, Strukturen und Prozesse 04.03.02

In [8]:
D_Governance_Strukturen_Prozesse={'Begründung, warum ist es ein Projekt':'',
                                  'Projektart und Begründung':'',
                                  'Klassufizierung aus Sicht der Organisation':'',
                                  'Strukturen der Organisation':''}

Infos from chapt 3. Anforderungen und Ziele 4.5.2.

In [9]:
D_Steckbrief={'Projektbezeichnung':'', 
              'Projektnummer':'',
              'Auftraggeber':'',
              'Projektleitung, Verantwortung':'',
              'Leistung':None,
              'Termine':None,
              'Kosten':None,
              'Projektorganisationsform':'',
              'Stabs-PO':None,
              'Matrix-PO':None,
              'Reine_PO':None,
              'Oberziel':'',
              'Projektinhalt':'',
              'Projektbeteiligte':'',
              'Projektumfeld':'',
              'Starttermin':'',
              'Endetermin': '',
              'Dauer': '',
              'Zwischentermine':'',
              'Aufwand-gesamt':'',
              'Aufwand-intern':'',
              'Aufwand-extern':'',
              'Personalkosten-gesamt':'',
              'Personalkosten-intern':'',
              'Personalkosten-extern':'',
              'Sachkosten-gesamt':'',
              'Sachkosten-intern':'',
              'Sachkosten-extern':'',
              'Investitionen-gesamt':'',
              'Investitionen-intern':'',
              'Investitionen-extern':'',
              'Budget-gesamt':'',
              'Budget-intern':'',
              'Budget-extern':'',
              'vrsl. Behinderungen/Risiken/Störungen':'',
              'Kunde':'',
              'Abnahmekriterien':''}

D_Oberziel={'Bezeichnung':'', 'Beschreibung':'', 'Messkriterium':''}

#Ergebnisziele:
D_Finanzziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Leistungsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_Qualitätsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}        #Further entries are added later on 
D_SozialeZiele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}          #Further entries are added later on 

#Vorgehensziele:
D_Terminziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Kostenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}           #Further entries are added later on 
D_Aufwandsziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}         #Further entries are added later on 
D_Projektrahmenziele={'1-Bezeichnung':'', '1-Beschreibung':'', '1-Messkriterium':''}    #Further entries are added later on 

#Nicht-Ziele:
D_NichtZiele={'1-Bezeichnung':'', '1-Beschreibung':''}                                  #Further entries are added later on 

#Zielkonflikte
D_Zielkonflikte={'1-Ziel_a':'',
                 '1-Ziel_b':'',
                 '1-Konfliktart':'',
                 '1-Priorität':'',
                 '1-Erklärung':'',
                 '1-Massnahmen':''} #Further entries are added later on 

Infos from chapt 4. Stakeholder 4.5.12.

In [10]:
D_Projektumfeld={'sozial-intern':'',
                 'sozial-extern':'',
                 'sachlich-intern':'',
                 'sachlich-extern':''}

D_Schnittstellen_Stakeholder_Umfeld={'1-Schnittstelle':'',
                                     '1-Ansprechpartner':'',
                                     '1-Auswirkung':'',
                                     '2-Schnittstelle':'',
                                     '2-Ansprechpartner':'',
                                     '2-Auswirkung':''} #Further entries may be added later on 

D_Stakeholderinteressen={'1-Stakeholder':'',
                         '1-Konfliktpotentzial':'',
                         '1-Einfluss/Macht':'',
                         '1-Erwartungen/Befrüchtungen':'',
                         '1-Strategie':'',
                         '1-Steuerung/Massnahmen':''} #Further entries are added later on 

Infos in chapt. 5. Macht und Interessen 04.03.04

In [11]:
#Maybe fuse into one dictionary
D_Stakeholderbewertung={'1-Stakeholder':'',
                        '1-Macht':'',
                        '1-Begründung der Machtposition':''} #Further entries are added later on 

D_Machtpromotoren={'Machtpromotoren':''}

Prints all text blocks

In [12]:
past_block=''
for block in iter_block_items(doc):
    current_block=block.text
    # ignore repeated cells, #ignore paragraphs starting with ignore_list
    if current_block != past_block: #and not current_block.startswith(ignore_list[0]) and not current_block.startswith(ignore_list[1]) and not current_block.startswith(ignore_list[2]) and not current_block.startswith(ignore_list[3]) and not current_block.startswith(ignore_list[4])
        print(block.text)
        #print(block.style)
    past_block=current_block

Report Level D

nach Z01D_Leitfaden / 02 vom 20.06.2023

Dieses Dokument basiert auf der Report-Vorlage der Decisio Projekt- und Prozessmanagement GmbH. 

Projekt:
Servicemigration IT-Servicecenter
Teilnehmer:
JT
Firma:
ABC
Adresse:

Telefon:

E-Mail:

Kurs:
23-1111

Änderungshistorie	
Version
Datum
Ersteller
Grund
0.1
26.09.23
JT
Ersterstellung
0.2
26.11.23
JT
Überarbeitung
1.0
tt.mm.jj
Autor
Fertigstellung

Hinweis zur Bearbeitung

Zum Abschluss, kurz vor dem Upload, bitte die Versionsnummer in der Fußzeile anpassen.

Inhaltsverzeichnis
1.	Strategie 4.5.1.	6
1.1.	Beschreibung des Business Case zum Projekt	6
1.2.	Nennung der kritischen Erfolgsfaktoren des Projekts	6
2.	Governance, Strukturen und Prozesse 04.03.02	7
2.1.	Begründung, warum es sich bei dem Vorhaben um ein Projekt handelt	7
2.2.	Nennung der Projektart des Projektes und Begründung der Einordnung	7
2.3.	Nennung und Begründung der Klassifizierung des Projekts aus Sicht der Organisation	7
2.4.	Nennung der Strukturen der Organ

Function to clear all auxiliary search variables

In [17]:
def clear_search_variables():
    textvar=""
    past_block=""
    current_block=""
    found_Chapter=False
    found_Cell=False

TO DO


Saves the infos of the first table in the cover page

TO DO 

Saves Änderungshistorie table

TO DO

Saves Angaben zum Projekt and Eigene Position

### CHAPTER 1: Strategie 4.5.1. 

Looks for paragraphs in Subchapter 1.1 'Beschreibung des Business Case zum Projekt'  and saves them into a directory

In [18]:
# The text of the chapter will be saved into this variable
textvar=""
past_block=''
found_Chapter=False
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Beschreibung des Business Case zum Projekt'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
    #print(block.style.name)
    past_block=current_block
D_Strategie['Beschreibung des Business Case zum Projekt']=textvar
print(D_Strategie['Beschreibung des Business Case zum Projekt'])
clear_search_variables()

Das beschriebene Projekt weist einen besonderen Nutzen vor, wenn das fixe Gesamtbudget des Projektes in Höhe von 120.485,02 € nicht überschritten wird und darüber hinaus alle Projektziele innerhalb des ZeABClanes erreicht werden.Die monatlichen Kosten des Servicecenters der ABC GmbH belaufen sich auf 32.000 €, die gegen-über der monatlichen Kosten des IT Betriebes der Beckedorfer Bank in Höhe von 46.000 €, zu ei-ner Reduzierung von 14.000 € führen. Demnach wird sich eine Amortisation innerhalb von 13 Mo-naten einstellen.Weitere Nutzen stellen sich durch die Übernahme von Aufgaben einschließlich der Erfüllung von aufsichtsrechtlichen Anforderungen durch die ABC GmbH, die Minimierung von Betriebsrisiken, der Zugang zu externem Spezialistenwissen, Effizienzsteigerungen und Qualitätsverbesserungen und einem flexiblen Kostenmodell dar, welches eine bedarfsgerechte Skalierung ermöglicht.


Looks for paragraphs in Subchapter 1.2 'Nennung der kritischen Erfolgsfaktoren des Projekts' and saves them into a directory

In [19]:
# The text of the chapter will be saved into this variable
textvar=""
past_block=''
found_Chapter=False
for block in iter_block_items(doc):
    current_block=block
    # don't need to ignore repeated text from combined cells for this case
    #Are we in the section we're looking for based on a previous block?
    if past_block!='' and past_block.style.name=='Heading 2' and past_block.text.startswith('Nennung der kritischen Erfolgsfaktoren des Projekts'):
        found_Chapter=True
    #Break argument. When are we leaving the section we're lookin for?
    if found_Chapter:
        if block.style.name=='Heading 2' or block.style.name=='Heading 1':
            break
        textvar+=block.text
    #print(block.style.name)
    past_block=current_block
D_Strategie['Nennung der kritischen Erfolgsfaktoren des Projekts']=textvar
print(D_Strategie['Nennung der kritischen Erfolgsfaktoren des Projekts'])
clear_search_variables()

Als kritische Erfolgsfaktoren konnten für dieses Projekt identifiziert werden:Damit das Projekt überhaupt starten kann, muss der Migrationsvertrag vor dem offiziellen Start zum 01.09.22 vorliegen. Dieser wird als Projektauftrag gesehen.Die Vorlage des Betriebsvertrages ist für den Betriebsübergang (Cut Over) notwendig, da dieser sonst offiziell nicht durchgeführt werden darf. Hierzu erfolgt eine frühzeitige Abstimmung mit dem Kunden und Anlauf der offiziellen Prozesse.Die Projektergebnisse werden in die Linie übergeben, die in das Projekt eingebunden wer-den. Die Projektergebnisse werden standardisiert.Bestehende Hardware läuft während des Projektes aus dem EOS und muss innerhalb von 2 Monaten getauscht werden. Durch frühzeitige Kontrolle und Sensibilisierung der Beckedorfer Bank erfolgt ein Austausch aller Systeme vor dem Cut Over.Mitarbeiter der Bank sind im Besitz von Administrationsrechten, die zum Betriebsübergang abgegeben werden müssen, da die Verantwortung ab diesem ZeABCunkt b